In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


import numpy as np
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import random
from scipy.io import mmread
import math
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import os
from matplotlib import gridspec
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import chi2,f_classif,mutual_info_classif,mutual_info_regression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from scipy.stats import chisquare
from scipy.stats import pearsonr
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from parfor import parfor
import time
from time import sleep
from parfor import parfor
from scipy import io as sio
from scipy import sparse as ss
from scipy import optimize as so
from sklearn.utils.sparsefuncs import mean_variance_axis
from scipy.stats import linregress
import csv
import seaborn as sns
import copy
from sklearn.cluster import KMeans
import pickle
import numpy as np
import pandas as pd

In [2]:
def text_create(path, name, msg):
    full_path = path + "/" + name + '.txt'
    file = open(full_path, 'w')
    file.write(str(msg))

def SVM(X, y):
    model = svm.LinearSVC(max_iter=1000000)
    model.fit(X, y)
    return model


def get_error(model, X, y):
    y_pred = model.predict(X)
    return mean_squared_error(y_pred, y)

def feature_compare(path,X, Y, X_test,Y_test,num_features, num_samples_list):
    train_error=[]
    test_error=[]
    train_acc=[]
    test_acc=[]
    D = np.shape(X)[1]
    # Random
    ran_idx = random.choices(range(D), k=num_features)
    text_create(path,'ran_idx',ran_idx)
    for i in range(num_features):
        model=SVM(X[:,ran_idx[:(i+1)]],Y)
        train_error.append(get_error(model,X[:,ran_idx[:(i+1)]],Y))
        train_acc.append(model.score(X[:,ran_idx[:(i+1)]],Y))
        model=SVM(X_test[:,ran_idx[:(i+1)]],Y_test)
        test_error.append(get_error(model,X_test[:,ran_idx[:(i+1)]],Y_test))
        test_acc.append(model.score(X_test[:,ran_idx[:(i+1)]],Y_test))
    print('ran_idx:')
    print(ran_idx)
    
    # correlation coefficient 
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(f_classif, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    cor_idx=[]
    for i in range(num_features):
        cor_idx.append(tmp[i][:(i+1)])
    text_create(path,'cor_idx',cor_idx)
    for i in range(num_features):
        model=SVM(X[:,cor_idx[i]],Y)
        train_error.append(get_error(model,X[:,cor_idx[i]],Y))
        train_acc.append(model.score(X[:,cor_idx[i]],Y))
        model=SVM(X_test[:,cor_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,cor_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,cor_idx[i]],Y_test))
    print('cor_idx:')
    print(cor_idx)
    
    # mutual
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(mutual_info_classif, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    mut_idx=[]
    for i in range(num_features):
        mut_idx.append(tmp[i][:(i+1)])
    text_create(path,'mut_idx',mut_idx)
    for i in range(num_features):
        model=SVM(X[:,mut_idx[i]],Y)
        train_error.append(get_error(model,X[:,mut_idx[i]],Y))
        train_acc.append(model.score(X[:,mut_idx[i]],Y))
        model=SVM(X_test[:,mut_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,mut_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,mut_idx[i]],Y_test))
    print('mut_idx:')
    print(mut_idx)
    
    # chi2
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = SelectKBest(chi2, k=D).fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        score_raw = np.abs(model.scores_).tolist()
        score = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in score_raw]
        idx = model.get_support(indices=True).tolist()
        return [x for _, x in sorted(zip(score, idx), reverse=True)]
    chi_idx=[]
    for i in range(num_features):
        chi_idx.append(tmp[i][:(i+1)])
    text_create(path,'chi_idx',chi_idx)
    for i in range(num_features):
        model=SVM(X[:,chi_idx[i]],Y)
        train_error.append(get_error(model,X[:,chi_idx[i]],Y))
        train_acc.append(model.score(X[:,chi_idx[i]],Y))
        model=SVM(X_test[:,chi_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,chi_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,chi_idx[i]],Y_test))
    print('chi_idx:')
    print(chi_idx)
    
    # Feature importance is an inbuilt class that comes with Tree Based Classifiers
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model = ExtraTreesClassifier()
        model.fit(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        coef = [float('-inf') if math.isnan(x) or x==float('inf') else x for x in np.abs(model.feature_importances_)]
        return sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)
    tree_idx=[]
    for i in range(num_features):
        tree_idx.append(tmp[i][:(i+1)])
    text_create(path,'tree_idx',tree_idx)
    for i in range(num_features):
        model=SVM(X[:,tree_idx[i]],Y)
        train_error.append(get_error(model,X[:,tree_idx[i]],Y))
        train_acc.append(model.score(X[:,tree_idx[i]],Y))
        model=SVM(X_test[:,tree_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,tree_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,tree_idx[i]],Y_test))
    print('tree_idx:')
    print(tree_idx)
    
    # SVM by weight
    @parfor(range(num_features), bar=False)
    def tmp(i):
        indices=np.arange(np.shape(X)[0])
        random.shuffle(indices)
        model=SVM(X[indices[:num_samples_list[i]]], Y[indices[:num_samples_list[i]]])
        weight=np.sum(np.abs(model.coef_),axis=0).tolist()
        coef = [0 if math.isnan(x) or x==float('inf') else x for x in weight]
        return sorted(range(len(coef)), key=lambda i: coef[i], reverse=True)
    svm_idx=[]
    for i in range(num_features):
        svm_idx.append(tmp[i][:(i+1)])
    text_create(path,'svm_idx',svm_idx)
    for i in range(num_features):
        model=SVM(X[:,svm_idx[i]],Y)
        train_error.append(get_error(model,X[:,svm_idx[i]],Y))
        train_acc.append(model.score(X[:,svm_idx[i]],Y))
        model=SVM(X_test[:,svm_idx[i]],Y_test)
        test_error.append(get_error(model,X_test[:,svm_idx[i]],Y_test))
        test_acc.append(model.score(X_test[:,svm_idx[i]],Y_test))
    print('svm_idx:')
    print(svm_idx)
    text_create(path,'compare_train_error',train_error)
    text_create(path,'compare_test_error',test_error)
    text_create(path,'compare_train_acc',train_acc)
    text_create(path,'compare_test_acc',test_acc)
    return train_error,test_error,train_acc,test_acc,ran_idx,cor_idx,mut_idx,chi_idx,tree_idx,svm_idx



In [3]:
data_df = pd.read_csv('./data/PBMCnorm_final.csv')
label_df=pd.read_csv('./data/labels_final.csv',header = None)
gene_df=pd.read_csv('./data/genes_final.csv')
classes=[0,1,2,3,4]
keys=['Monocytes','T cells','Act. T/NK','B cells','Others']

raw_data=data_df.values
target=label_df.values.reshape((raw_data.shape[1],)).astype(np.uint8)-1
gene=gene_df.values.reshape((raw_data.shape[0],))
data=normalize(np.transpose(raw_data),axis=1, norm='l2')
del data_df,label_df,gene_df,raw_data


idx = np.arange(np.shape(data)[0])
random.shuffle(idx)
X_train = data[idx[:int(np.shape(data)[0]*4/5)],:]
y_train = target[idx[:int(np.shape(data)[0]*4/5)]]
X_test = data[idx[int(np.shape(data)[0]*4/5):],:]
y_test = target[idx[int(np.shape(data)[0]*4/5):]]

print(type(data))
print(np.shape(data),np.shape(target),len(np.unique(target)))
print(np.shape(X_train))
print(np.shape(X_test))
for i in np.unique(target):
    print('class '+keys[i]+': '+str(np.count_nonzero((target==i)*1)))

<class 'numpy.ndarray'>
(10194, 6915) (10194,) 5
(8155, 6915)
(2039, 6915)
class Monocytes: 3269
class T cells: 3517
class Act. T/NK: 1265
class B cells: 995
class Others: 1148


In [4]:
# You can ignore all warnings here
num_features = 100
num_samples_list=[200, 200, 212, 214, 245, 248, 249, 252, 261, 261, 275, 281, 281, 286, 286, 289, 291, 294, 298, 298, 301, 303, 305, 306, 306, 306, 306, 306, 306, 306, 306, 306, 306, 307, 307, 308, 308, 308, 308, 308, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 309, 312, 312, 312, 315, 316, 316, 318, 319, 319, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 321, 321, 321, 321, 321, 321, 321, 322, 322, 322, 322, 322, 322]


path='results/compare_mincell'
try:
    os.mkdir('results')
except OSError:
    print ("Creation of the directory %s failed")
else:
    print ("Successfully created the directory %s ")
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

train_error,test_error,train_acc,train_acc,ran_idx,fvalue_idx,mut_idx,chi_idx,tree_idx,svm_idx=feature_compare(path,X_train, y_train, X_test,y_test,num_features, num_samples_list)



Creation of the directory %s failed
Successfully created the directory results/compare_mincell 
ran_idx:
[1195, 781, 2372, 715, 2344]


/Users/xqchen/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   0    3   12 ... 6905 6911 6913] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/xqchen/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/xqchen/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   0    1    3 ... 6912 6913 6914] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/xqchen/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/Users/xqchen/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning:

cor_idx:
[[3893], [1863, 2779], [2779, 5069, 1717], [4017, 3893, 4659, 2779], [3893, 2619, 3894, 5015, 3721]]
mut_idx:
[[2323], [430, 2025], [2323, 2230, 3758], [2025, 2230, 3893, 4376], [2230, 2323, 3893, 4376, 1097]]
chi_idx:
[[4719], [6118, 3721], [6203, 3721, 6118], [3721, 4718, 6118, 1717], [4719, 3721, 6203, 6118, 870]]
tree_idx:
[[6064], [6756, 5350], [3893, 979, 4488], [563, 488, 3895, 6118], [3702, 6203, 3893, 2779, 3895]]
svm_idx:
[[5475], [5475, 1829], [6203, 5475, 870], [6203, 870, 1829, 5475], [5475, 1829, 6203, 870, 1863]]
